# TP Final Integrador Visión por Computadora II - CEIA - FIUBA - Cohorte 16

Alumnos: Fabricio Lopretto (a1616) y Santiago José Olaciregui (a1611)

## Ingeniería de características sobre el conjunto de datos

### Objetivos:

Preparar los datos para ser utilizados por los modelos de detección de objetos propuestos, realizando las siguientes operaciones:

1. Redimensionado: Ajustar las dimensiones para que sean uniformes, manteniendo relaciones de aspecto o usando recortes.
2. Normalización: Escalar valores de píxeles entre 0 y 1 (o estandarizar con media cero y desviación estándar uno).
3. Ajuste de Color: Convertir entre espacios de color (RGB a HSV, escala de grises) ayuda en tareas específicas, como detección de bordes.
4. Blurred Backgrounds o Masking: son técnicas utilizadas para resaltar el sujeto o un área específica en una imagen al modificar el fondo o aplicarle máscaras.

### 1 - Redimensionado

*Nota: las imagenes ya presentan una uniformidad de dimensiones, son todas de 416x416 píxeles.*

In [1]:
# Importa librerías necesarias
import cv2
import os
import numpy as np

In [2]:
# Directorio de la notebook
script_path = os.getcwd()

In [ ]:
def resize_with_aspect_ratio(image, target_size=(416, 416)):
    """
    Redimensiona la imagen para ajustarse a las dimensiones objetivo,
    manteniendo la relación de aspecto y agregando relleno.
    """
    h, w = image.shape[:2]
    scale = min(target_size[1] / w, target_size[0] / h)
    new_w, new_h = int(w * scale), int(h * scale)

    resized_image = cv2.resize(image, (new_w, new_h), interpolation=cv2.INTER_AREA)
    delta_w, delta_h = target_size[1] - new_w, target_size[0] - new_h

    top, bottom = delta_h // 2, delta_h - (delta_h // 2)
    left, right = delta_w // 2, delta_w - (delta_w // 2)
    
    color = [0, 0, 0]  # Color negro para el relleno
    padded_image = cv2.copyMakeBorder(resized_image, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)
    
    return padded_image

def process_images(input_folder, output_folder, target_size=(416, 416)):
    """
    Procesa todas las imágenes en la carpeta de entrada,
    redimensionándolas y guardándolas en la carpeta de salida.
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
        
    for filename in os.listdir(input_folder):
        image_path = os.path.join(input_folder, filename)
        image = cv2.imread(image_path)

        if image is not None:
            resized_image = resize_with_aspect_ratio(image, target_size)
            output_path = os.path.join(output_folder, filename)
            cv2.imwrite(output_path, resized_image)
            print(f"Procesado: {filename}")


# Directorio de conjunto de datos de entrada
input_folder_train = script_path + '/data/data_original/train/images/'
# Directorio de conjunto de datos de salida
output_folder_train = script_path + '/data/data_procesada/train_mask_background/images/'

# llamada a la función
process_images(input_folder_train, output_folder_train)

### 2 - Normalización

In [ ]:
def normalize_image(image, method='scaling'):
    """
    Normaliza la imagen.
    method: 'scaling' para escalar entre 0 y 1, 'standardization'
    para estandarizar a media cero y desviación estándar uno.
    """
    if method == 'scaling':
        normalized_image = image / 255.0  # Escala valores de píxeles entre 0 y 1
    elif method == 'standardization':
        mean, std = image.mean(), image.std()
        normalized_image = (image - mean) / std
    else:
        raise ValueError("Método no soportado. Usa 'scaling' o 'standardization'.")
    
    return normalized_image

def process_normalization(input_folder, output_folder, method='scaling'):
    """
    Lee imágenes de la carpeta redimensionada, aplica normalización,
    y guarda los resultados.
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for filename in os.listdir(input_folder):
        image_path = os.path.join(input_folder, filename)
        image = cv2.imread(image_path)
        
        if image is not None:
            normalized_image = normalize_image(image, method)
            output_path = os.path.join(output_folder, filename)
            
            # Multiplicar por 255 para guardar la imagen en formato de 8 bits (si es necesario)
            cv2.imwrite(output_path, (normalized_image * 255).astype(np.uint8))
            print(f"Imagen normalizada y guardada: {filename}")


# Directorio de conjunto de datos de entrada
input_folder_train = script_path + '/data/data_procesada/train_mask_background/images/'
# Directorio de conjunto de datos de salida
output_folder_train = script_path + '/data/data_procesada/train_mask_background/images/'

 # Llamada a la función: Usa 'standardization' para estandarización
process_normalization(input_folder_train, output_folder_train, method='scaling')

Imagen normalizada y guardada: 000244709_jpg.rf.6ad3fbffe1283e6d8f48a207aecfb9cd.jpg
Imagen normalizada y guardada: 000390579_jpg.rf.ecc8f1090e6d61590576896ba79f5a3c.jpg
Imagen normalizada y guardada: 000413060_jpg.rf.c6da079ccf2c5b108ef52301919c5576.jpg
Imagen normalizada y guardada: 000720153_jpg.rf.9b5cc2fe2de88da09c97f44658ddbf68.jpg
Imagen normalizada y guardada: 000782346_jpg.rf.9a65bbf59d8b976c5b26a701ffe57569.jpg
Imagen normalizada y guardada: 000796761_jpg.rf.c7bdeba0534b24b20c5b11aac872c39a.jpg
Imagen normalizada y guardada: 000809418_jpg.rf.029957bbb49844c87bcd4f439438e167.jpg
Imagen normalizada y guardada: 001947084_jpg.rf.707c573a3c876c1eb6d32ed1cd25126b.jpg
Imagen normalizada y guardada: 002066279_jpg.rf.13e0a60f7702193ceaba6be107887e5a.jpg
Imagen normalizada y guardada: 002517602_jpg.rf.3cd96925711b565084ea7e5ed93a5574.jpg
Imagen normalizada y guardada: 002647452_jpg.rf.5e135efa0db715e39ef309b5d0b12336.jpg
Imagen normalizada y guardada: 002734491_jpg.rf.96562251283efc45f

### 3 - Ajuste de Color a escala de grises

*Nota: Se realiza la escala de grises dado que solo interesa el número y no el color (los números son todos blancos), con lo cual se puede simplificar y reducir la dimensionalidad mediante esta técnica.*

In [ ]:
def convert_to_grayscale(image):
    """
    Convierte la imagen a escala de grises usando OpenCV.
    """
    # Convierte la imagen a escala de grises
    grayscale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return grayscale_image

def process_grayscale(input_folder, output_folder):
    """
    Procesa todas las imágenes aplicando la conversión
    y guardando las nuevas imágenes en la carpeta de salida.
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for filename in os.listdir(input_folder):
        image_path = os.path.join(input_folder, filename)
        image = cv2.imread(image_path)
        
        if image is not None:
            grayscale_image = convert_to_grayscale(image)
            output_path = os.path.join(output_folder, filename)
            cv2.imwrite(output_path, grayscale_image)
            print(f"Imagen convertida a escala de grises y guardada: {filename}")


# Directorio de conjunto de datos de entrada
input_folder_train = script_path + '/data/data_procesada/train_mask_background/images/'
# Directorio de conjunto de datos de salida
output_folder_train = script_path + '/data/data_procesada/train_mask_background/images/'

# llamada a la función
process_grayscale(input_folder_train, output_folder_train)

Imagen convertida a escala de grises y guardada: 000244709_jpg.rf.6ad3fbffe1283e6d8f48a207aecfb9cd.jpg
Imagen convertida a escala de grises y guardada: 000390579_jpg.rf.ecc8f1090e6d61590576896ba79f5a3c.jpg
Imagen convertida a escala de grises y guardada: 000413060_jpg.rf.c6da079ccf2c5b108ef52301919c5576.jpg
Imagen convertida a escala de grises y guardada: 000720153_jpg.rf.9b5cc2fe2de88da09c97f44658ddbf68.jpg
Imagen convertida a escala de grises y guardada: 000782346_jpg.rf.9a65bbf59d8b976c5b26a701ffe57569.jpg
Imagen convertida a escala de grises y guardada: 000796761_jpg.rf.c7bdeba0534b24b20c5b11aac872c39a.jpg
Imagen convertida a escala de grises y guardada: 000809418_jpg.rf.029957bbb49844c87bcd4f439438e167.jpg
Imagen convertida a escala de grises y guardada: 001947084_jpg.rf.707c573a3c876c1eb6d32ed1cd25126b.jpg
Imagen convertida a escala de grises y guardada: 002066279_jpg.rf.13e0a60f7702193ceaba6be107887e5a.jpg
Imagen convertida a escala de grises y guardada: 002517602_jpg.rf.3cd9692

### 4 - Blurred Backgrounds o Masking

*Nota: Aplica desenfoque (filtro gaussiano) al fondo para que el modelo se concentre en el objeto principal (números de las cartas).*

In [ ]:
def apply_blurred_background(image):
    """
    Utiliza el umbral para segmentar el área de las cartas.
    Aplica un desenfoque Gaussiano al fondo.
    """
    # Aplica umbral para segmentar el objeto (las cartas) en una imagen en escala de grises
    _, mask = cv2.threshold(image, 100, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Crea el fondo desenfocado
    blurred_background = cv2.GaussianBlur(image, (41, 41), sigmaX=50)

    # Enfoca el objeto (las cartas) y aplica el fondo desenfocado
    focus_object = cv2.bitwise_and(image, mask)
    background = cv2.bitwise_and(blurred_background, cv2.bitwise_not(mask))
    result = cv2.add(focus_object, background)

    return result

def process_blurred_background(input_folder, output_folder):
    """
    Mantiene las cartas enfocadas y el fondo desenfocado.
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for filename in os.listdir(input_folder):
        image_path = os.path.join(input_folder, filename)
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        
        if image is not None:
            result_image = apply_blurred_background(image)
            output_path = os.path.join(output_folder, filename)
            cv2.imwrite(output_path, result_image)
            print(f"Imagen con fondo desenfocado guardada: {filename}")


# Directorio de conjunto de datos de entrada
input_folder_train = script_path + '/data/data_procesada/train_mask_background/images/'
# Directorio de conjunto de datos de salida
output_folder_train = script_path + '/data/data_procesada/train_mask_background/images/'

# Llamada a la función
process_blurred_background(input_folder_train, output_folder_train)

Imagen con fondo desenfocado guardada: 000244709_jpg.rf.6ad3fbffe1283e6d8f48a207aecfb9cd.jpg
Imagen con fondo desenfocado guardada: 000390579_jpg.rf.ecc8f1090e6d61590576896ba79f5a3c.jpg
Imagen con fondo desenfocado guardada: 000413060_jpg.rf.c6da079ccf2c5b108ef52301919c5576.jpg
Imagen con fondo desenfocado guardada: 000720153_jpg.rf.9b5cc2fe2de88da09c97f44658ddbf68.jpg
Imagen con fondo desenfocado guardada: 000782346_jpg.rf.9a65bbf59d8b976c5b26a701ffe57569.jpg
Imagen con fondo desenfocado guardada: 000796761_jpg.rf.c7bdeba0534b24b20c5b11aac872c39a.jpg
Imagen con fondo desenfocado guardada: 000809418_jpg.rf.029957bbb49844c87bcd4f439438e167.jpg
Imagen con fondo desenfocado guardada: 001947084_jpg.rf.707c573a3c876c1eb6d32ed1cd25126b.jpg
Imagen con fondo desenfocado guardada: 002066279_jpg.rf.13e0a60f7702193ceaba6be107887e5a.jpg
Imagen con fondo desenfocado guardada: 002517602_jpg.rf.3cd96925711b565084ea7e5ed93a5574.jpg
Imagen con fondo desenfocado guardada: 002647452_jpg.rf.5e135efa0db715

### Observaciones:

Producto de estas operaciones, el conjunto de imagenes incrementó su peso en un 11,2%.